### Verify Processed

In [1]:
import pandas as pd

interim_path = 'data/test/translated_ds_011.parquet'

df = pd.read_parquet(interim_path)
print(f'total de linhas: {len(df)}')


total de linhas: 1342


In [2]:
df.messages_translated.sample(1).values[0]

array([{'messageId': 220812, 'senderWorkerId': 1057, 'text': 'Oi', 'timeOffset': 0},
       {'messageId': 220813, 'senderWorkerId': 1057, 'text': 'Como você está?', 'timeOffset': 4},
       {'messageId': 220816, 'senderWorkerId': 1049, 'text': 'Olá', 'timeOffset': 14},
       {'messageId': 220817, 'senderWorkerId': 1049, 'text': 'Qual é o seu filme favorito?', 'timeOffset': 45},
       {'messageId': 220821, 'senderWorkerId': 1057, 'text': 'Ou @134106 ou @148871.', 'timeOffset': 79},
       {'messageId': 220824, 'senderWorkerId': 1057, 'text': '@187028 Foi bom também.', 'timeOffset': 107},
       {'messageId': 220830, 'senderWorkerId': 1049, 'text': 'Você já viu o @86397? Além disso, o @100646 é um bom filme.', 'timeOffset': 203},
       {'messageId': 220831, 'senderWorkerId': 1057, 'text': 'Eu gostei de Batman Forever, nunca vi Deep Impact.', 'timeOffset': 228},
       {'messageId': 220832, 'senderWorkerId': 1057, 'text': 'Vou ter que dar uma olhada', 'timeOffset': 231},
       {'messa

### Example generating batchs samples

In [3]:
movies = pd.DataFrame(df['movieMentions'].explode().to_list()).drop_duplicates()

In [4]:
import re
def id2text(text):
    for movie_id in re.findall(r'@\d+', text):
        movie_name = movies[movies['movieId'] == movie_id[1:]]['movieName'].iloc[0]
        text = text.replace(movie_id, movie_name)
    return text

In [5]:
df_dev = df.head()
df_dev = df_dev[['messages_translated']]
df_dev['conversation'] = df_dev['messages_translated'].apply(lambda x: [id2text(message['text']) for message in x])
df_dev.drop(columns=['messages_translated'], inplace=True)

In [6]:
# split question x response explode
df_train = []

for _, row in df_dev.iterrows():
    for i in range(len(row['conversation']) - 1):
        df_train.append({'question': row['conversation'][i], 'response': row['conversation'][i+1]})

df_train = pd.DataFrame(df_train)
df_train

,question,response
0,"Olá, estou procurando por um filme como o Supe...",Você deveria assistir Police Academy (1984).
1,Você deveria assistir Police Academy (1984).,É um grande? Eu nunca vi isso. Eu já vi Americ...
2,É um grande? Eu nunca vi isso. Eu já vi Americ...,Eu quero dizer American Pie (1999)
3,Eu quero dizer American Pie (1999),"Sim, Police Academy (1984) é muito engraçado ..."
4,"Sim, Police Academy (1984) é muito engraçado ...",Parece que eu preciso dar uma olhada neles.
...,...,...
74,Acho que vou tentar esse.,X-Men: The Last Stand (2006) Foi outro bom fil...
75,X-Men: The Last Stand (2006) Foi outro bom fil...,"Okay, obrigado!"
76,"Okay, obrigado!","Oh, tem tantas coisas"
77,"Oh, tem tantas coisas",Obrigado.


In [8]:
df_dev.iloc[0].values[0]

['Olá, estou procurando por um filme como o Super Troopers (2001).',
 'Você deveria assistir Police Academy  (1984).',
 'É um grande? Eu nunca vi isso. Eu já vi American Pie .',
 'Eu quero dizer American Pie  (1999)',
 'Sim, Police Academy  (1984) é muito engraçado e Police Academy 2: Their First Assignment (1985) também.',
 'Parece que eu preciso dar uma olhada neles.',
 'Sim, você vai gostar deles.',
 'Eu agradeço seu tempo. Eu precisarei dar uma olhada nisso. Existem outros que você recomendaria?',
 'Sim Lethal Weapon (1987)',
 'Obrigado, eu também vou assistir isso.',
 'E também Beverly Hills Cop (1984)',
 'Obrigado pelas sugestões.',
 'Você é bem-vindo',
 'E também 48 Hrs. (1982)',
 'Obrigado, adeus.']

In [10]:
df_dev.style

,conversation
0,"['Olá, estou procurando por um filme como o Super Troopers (2001).', 'Você deveria assistir Police Academy (1984).', 'É um grande? Eu nunca vi isso. Eu já vi American Pie .', 'Eu quero dizer American Pie (1999)', 'Sim, Police Academy (1984) é muito engraçado e Police Academy 2: Their First Assignment (1985) também.', 'Parece que eu preciso dar uma olhada neles.', 'Sim, você vai gostar deles.', 'Eu agradeço seu tempo. Eu precisarei dar uma olhada nisso. Existem outros que você recomendaria?', 'Sim Lethal Weapon (1987)', 'Obrigado, eu também vou assistir isso.', 'E também Beverly Hills Cop (1984)', 'Obrigado pelas sugestões.', 'Você é bem-vindo', 'E também 48 Hrs. (1982)', 'Obrigado, adeus.']"
1,"['Olá!', 'Olá!', 'Qual tipo de filmes você gosta?', 'Procuro uma recomendação de filme. Quando eu era mais jovem, eu realmente gostei muito do A Nightmare on Elm Street (1984).', 'Oh, você gosta de filmes de terror?', 'Recentemente, eu assisti Happy Death Day (2017).', 'Eu também gostei de assistir The Last House on the Left (1972)', 'Foi realmente bom para um novo ""filme de terror"".', 'Eu gosto de alguns dos filmes de terror mais recentes que eu vi também.', 'Ouvi dizer que o A Quiet Place (2018) é bom. Ele ainda está em cartaz nos cinemas, no entanto.', 'Eu gostei muito do filme.', 'Eu também!', 'Era realmente assustador, mas eu achei bom!', 'Eu vi esse filme no cinema, ele foi muito bom. Ele me deixou na ponta da cadeira durante todo o filme.', 'Não tenho certeza se vi aquilo, preciso verificar. Parece familiar, mas não tenho certeza. Obrigado pelas suas sugestões!', 'Existem algumas comédias que você sugere?', 'Claro! Eu gosto muito de comédias. Eu gosto de filmes como Hot Tub Time Machine e Super Troopers (2001), mas eu também gosto de Identity Thief (2013) e The Heat (2013).', 'Maravilhoso! Muito obrigado, acho que estou pronto para a noite de cinema agora.', 'Sem problemas! Obrigado também! :)']"
2,"['Me diga o que você gostaria de assistir?', 'Easy A (2010)', 'Você assistiu ao Avengers: Infinity War (2018)?', 'Or The Black Panthers: Vanguard of the Revolution (2015) ?', 'Eu realmente gostei disso.', 'Eu acho que eu gostaria de assistir Easy A (2010).', 'É engraçado', 'Mas você já sabe o que quer assistir', 'Você deveria me dizer que tipo de filmes você gosta.', 'Você assistiu ao Click (2006)?', 'Você gosta de filmes de terror?', 'Você assistiu ao It (2017)?', 'Sim, eu gosto do Adam Sandler. O The Waterboy (1998) é outro bom filme dele.', 'Eu não vi o filme.', 'Sim, acho que sim.', 'Espero ter ajudado.', 'Adeus']"
3,"['Eu gostaria de assistir a qualquer filme.', 'Me diga qualquer filme.', 'Gostei do seu post Avengers: Infinity War (2018).', 'Você viu The Avengers (2012)?', 'Eu realmente gostei daquele.', 'Sim e eu gostei disso.', 'Qual outro você me recomendaria?', 'Você gosta de coisas assustadoras?', 'Você viu It (2017)?', 'Eu assisti ao vídeo Iron Man 2 (2010) e gostei dele.', 'Ou Cabin by the Lake (2000)', 'Não, eu não vi It (2017).', 'Mas eu vou tentar.', 'Obrigado por me ajudar.', 'Adeus', 'Se você gosta de super-heróis, o Ant-Man (2015) também é uma ótima opção.', 'Adeus']"
4,"['Qual tipo de filmes você gosta?', 'Você viu o Avengers: Infinity War (2018)?', 'Ou Spider-Man (2002)', 'Eu também gostei deles.', 'Essas são incríveis.', 'Eu gosto desse tipo de filmes', 'X-Men: First Class (2011)', 'Eu também assisti Iron Man (2008).', 'Esses também são bons filmes.', 'Não, eu não assisti aquele.', 'Acho que vou tentar esse.', 'X-Men: The Last Stand (2006) Foi outro bom filme dos X-Men.', 'Okay, obrigado!', 'Oh, tem tantas coisas', 'Obrigado.', 'Adeus']"


In [12]:
def generate_sample(conversation):
    results = []
    for message in conversation:
        processed_example = "<|system|>\n Você é um chatbot de recomendação de filmes, converse com o usuário para indicar filmes apropriados.</s>\n<|user|>\n" + example[self.config.INSTRUCTION_FIELD] + "</s>\n<|assistant|>\n" + example[self.config.TARGET_FIELD]
